In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg


import global_settings
from utils import experiments, results
import jax
from multiprocessing import Pool
from tqdm import tqdm
import jax.numpy as jnp
import os
rng_key, rng_key_ = jax.random.split(jax.random.PRNGKey(0))
dataset_names = [
    'airfoil',
    'boston',
    'concrete',
    'diabetes',
    'energy',
    'forest_fire',
    'wine',
    'yacht'
]

In [3]:
experiment = experiments.ExperimentSampleStandard(
    settings=experiments.settings.SettingsExperimentSample(
        output_path=global_settings.PATH_PAPER_RESULTS,
        dataset=global_settings.DATASET_NAMES[0],
        dataset_normalization="standardization",
        hidden_layers=1,
        hidden_neurons=3,
        activation="tanh",
        activation_last_layer="none",
        num_warmup=2**10,
        statistic="reduced",
        statistic_p=0.99,
        samples_per_chain=1,
        identifiable_modes=3,
        pool_size=2,
        seed=0,
        overwrite_chains=None
    )
)

In [4]:
print("model transformation parameters {}".format(experiment._model_transformation.parameters_size(experiment._dataset[0][0])))

def run_parallel(key):
    experiment.sample(key)
    return experiment._mcmc.get_samples()

rng_key, *sub_keys = jax.random.split(jax.random.PRNGKey(experiment._settings.seed), experiment._sample_statistics["num_chains"] + 1)
with Pool(experiment._settings.pool_size) as p:
    samples_parallel = list(tqdm(p.imap(run_parallel, sub_keys), total=experiment._sample_statistics["num_chains"]))

samples = {}
for samples_run in samples_parallel:
    for key in samples_run.keys():
        if key not in samples:
            samples[key] = samples_run[key]
        else:
            samples[key] = jnp.concatenate([samples[key], samples_run[key]])
experiment._samples = samples

model transformation parameters 10


100%|█████████████████████████████████████████████████████████████████████████████████████| 1274/1274 [08:54<00:00,  2.38it/s]


In [5]:
print(f"saved as {experiment.save()}")

saved as /home/gw/data/experiments/paper/results/cd75123389f8789603a02650aaa042de.tar.gz
